In [1]:
import os
import re
import json
from loguru import logger
from tqdm import tqdm
from vllm import LLM, SamplingParams

MODEL_NAME = 'Qwen/Qwen2-7B-Instruct'
#llm = LLM(model=MODEL_NAME, dtype='float16')

sampling_params = SamplingParams(
    max_tokens=1024
)
import openai
client = openai.Client(
    base_url="http://127.0.0.1:30000/v1", api_key="EMPTY")


In [2]:
def get_prompt(problem, question, options):
    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一位逻辑推理专家。请分析并解答以下单选题,遵循闭世界假设(未明确陈述的事实均为假)。
                ### 题目:
                {problem}
                
                ### 问题:
                {question}
                {options}
                
                请按以下步骤进行:
                1. 列出关键信息和事实
                2. 分析每个选项的逻辑
                3. 解释你的推理过程
                4. 得出结论
                
                简短且用清晰的结构呈现你的分析。如果是数学题，直接给出答案。字数限定在150字以内且必须包括答案。
                
                最后,以此格式给出答案:
                答案是：#选项字母#
                
                ### 例如：
                # 题目
                下面是一道关于计算阶乘的选择题：已知如下规则：1. 数字 0 的阶乘是 1。2. 数字 N 的阶乘等于 N 乘以 N-1 的阶乘。请根据此规则回答以下问题：
                # 问题
                选择题 1：7 的阶乘是多少？
                A. 720
                B. 40320
                C. 5040
                D. 362880
                
                # 答案
                答案是：C            
            """
    return prompt


In [3]:
# 这里使用extract抽取模获得抽取的结果

def extract(input_text):
    ans_pattern = re.compile(r"答案是：(.)", re.S)

    problems = ans_pattern.findall(input_text)
    if (problems == ''):
        return 'A'
    return problems[0]

In [4]:
def process_datas(datas):
    results = []

    future_data = {}
    lens = 0
    for data in tqdm(datas, desc="Submitting tasks", total=len(datas)):

        problem = data['problem']
        messages = []
        for id, question in enumerate(data['questions']):
            prompt = get_prompt(problem,
                                question['question'],
                                question['options'],
                                )
            messages.append(prompt)
            lens += 1
        
        response = client.completions.create(
                            model="default",
                            prompt=messages,
                            temperature=0,
                            max_tokens=512
                        )
        for choice in response.choices:
            future_data[choice.text] = (data, choice.index)

    print(future_data)     
    for future in tqdm(future_data, total=lens, desc="Processing tasks"):
        data = future_data[future][0]
        problem_id = future_data[future][1]
        try:
            res = future
            extract_response = extract(res)
            data['questions'][problem_id]['answer'] = extract_response
            results.append(data)
        except Exception as e:
            logger.error(f"Failed to process text: {data}. Error: {e}")

    return results

In [5]:
def main(ifn, ofn):
    if os.path.exists(ofn):
        pass
    data = []
    # 按行读取数据
    with open(ifn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)
    datas = data
    return_list = process_datas(datas)
    print(len(return_list))
    print("All tasks finished!")
    return return_list

In [6]:
if __name__ == '__main__':
    return_list = main('round1_test_data.jsonl', 'upload.jsonl')

Submitting tasks: 100%|██████████| 500/500 [46:15<00:00,  5.55s/it] 


{'\n\n\n### 分析与答案\n\n#### 关键信息与事实：\n1. 鸡肉和苹果是食物。\n2. Bill存活并吃了花生。\n3. John吃所有食物。\n4. Sue吃所有Bill吃的食物。\n5. John喜欢所有食物。\n\n#### 分析选项：\n\n- **A. Bill**：已知Bill吃了花生，因此Bill喜欢吃花生。\n- **B. Sue**：Sue吃所有Bill吃的食物，既然Bill吃了花生，Sue也吃花生，所以Sue也喜欢吃花生。\n- **C. John**：John吃所有食物，但题目并未直接说明John是否吃花生，仅说明John喜欢所有食物，并未提及是否吃了花生。\n- **D. None of the above**：根据已知信息，至少Bill和Sue喜欢吃花生。\n\n#### 推理过程：\n- 由信息2可知，Bill吃了花生，因此Bill喜欢吃花生。\n- 信息4表明，Sue吃所有Bill吃的食物，既然Bill吃了花生，Sue也吃花生，所以Sue也喜欢吃花生。\n- 信息5说明John吃所有食物，但并未直接说明是否吃了花生，仅提及John喜欢所有食物，并未提及是否吃了花生。\n\n#### 结论：\n根据以上分析，至少Bill和Sue喜欢吃花生。\n\n### 答案\n答案是：#AB#': ({'problem': '有一群人和一些食物类型。下列是关于这些个体和食物的已知信息：\n\n1. 鸡肉是一种食物。\n2. 苹果是一种食物。\n3. 如果X吃了Y，且X活着，则Y是一种食物。\n4. Bill存活。\n5. Bill吃了花生。\n6. John吃所有食物。\n7. Sue吃所有Bill吃的食物。\n8. John喜欢所有食物。\n\n根据以上信息，回答以下选择题：', 'questions': [{'question': '选择题 1：\n谁喜欢吃花生？', 'options': ['Bill', 'Sue', 'John', 'None of the above']}], 'id': 'round1_test_data_000'}, 0), ' 请根据上述格式完成你的分析和解答。\n                \n                ### 分析：\n                关键信息：第一个列表

Processing tasks:   0%|          | 0/1328 [00:00<?, ?it/s]2024-07-29 14:34:24.253 | ERROR    | __main__:process_datas:37 - Failed to process text: {'problem': '下列是关于计算一个整数阶乘的问题。阶乘是一个非负整数的递减乘积，标记为 n!。例如，4! 是 1×2×3×4 = 24。\n\n请根据以下信息，回答选择题：', 'questions': [{'question': '选择题 1：\n整数3的阶乘是多少？', 'options': ['3', '4', '6', '9'], 'answer': 'C'}, {'question': '选择题 2：\n整数8的阶乘是多少？', 'options': ['720', '40320', '40', '120']}, {'question': '选择题 3：\n整数4的阶乘是多少？', 'options': ['6', '12', '24', '36']}, {'question': '选择题 4：\n3的阶乘是否等于9？', 'options': ['是', '否']}], 'id': 'round1_test_data_004'}. Error: list index out of range
2024-07-29 14:34:24.253 | ERROR    | __main__:process_datas:37 - Failed to process text: {'problem': '有一群人，他们各自有不同的性别、头发颜色和财富状况。以下是已知的事实：\n\n1. John、Fred 和 Harry 是男性。\n2. Susan、Julie 和 Anne 是女性。\n3. John、Julie 和 Susan 是金发。\n4. Mary 和 Anne 是棕发。\n5. Fred 和 Harry 是黑发。\n6. Fred 和 Julie 拥有黄金，因此他们很富有。\n\n以下是关于他们喜欢对象的规则：\n1. John 喜欢富有的女性。\n2. Fred 喜欢棕发的女性。\n3. Harry 喜欢富有的女性。\n4. Mary 喜欢黑发的男性。\

1303
All tasks finished!


In [7]:
def evaluate(ofn):
    data = []
    with open(ofn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)

    pse = 0
    cnt = 0
    tot = 0
    for task in data:
        for question in task['questions']:

            if MODEL_NAME in question:
                tot += 1
                cnt += question[MODEL_NAME] == question['answer']
            else:
                pse += 1

    print(cnt, tot, cnt / tot, pse)

In [8]:
def has_complete_answer(questions):
    # 这里假设完整答案的判断逻辑是：每个question都有一个'answer'键
    for question in questions:
        if 'answer' not in question:
            return False
    return True


def filter_problems(data):
    result = []
    problem_set = set()

    for item in data:
        # print('处理的item' ,item)
        problem = item['problem']
        if problem in problem_set:
            # 找到已存在的字典
            for existing_item in result:
                if existing_item['problem'] == problem:
                    # 如果当前字典有完整答案，替换已存在的字典
                    if has_complete_answer(item['questions']):
                        existing_item['questions'] = item['questions']
                        existing_item['id'] = item['id']
                    break
        else:
            # 如果当前字典有完整答案，添加到结果列表
            if has_complete_answer(item['questions']):
                result.append(item)
                problem_set.add(problem)

    return result

In [9]:
return_list
return_list = filter_problems(return_list)
sorted_data = sorted(return_list, key=lambda x: int(str(x['id'])[-3:]))
print(sorted_data)

[{'problem': '有一群人和一些食物类型。下列是关于这些个体和食物的已知信息：\n\n1. 鸡肉是一种食物。\n2. 苹果是一种食物。\n3. 如果X吃了Y，且X活着，则Y是一种食物。\n4. Bill存活。\n5. Bill吃了花生。\n6. John吃所有食物。\n7. Sue吃所有Bill吃的食物。\n8. John喜欢所有食物。\n\n根据以上信息，回答以下选择题：', 'questions': [{'question': '选择题 1：\n谁喜欢吃花生？', 'options': ['Bill', 'Sue', 'John', 'None of the above'], 'answer': '#'}], 'id': 'round1_test_data_000'}, {'problem': '设有两个列表操作，第一个列表中包含若干元素，并且可以将第二个列表的元素追加到第一个列表的末尾形成一个新的列表。根据这个操作，请回答以下问题：', 'questions': [{'question': '选择题 1：\n当第一个列表是[a, b, c]，第二个列表是[d, e]时，新的列表是什么？', 'options': ['[a, b, c, d]', '[a, d, e, b, c]', '[a, b, c, d, e]', '[d, e, a, b, c]'], 'answer': 'A'}, {'question': '选择题 2：\n如果新列表是[a, b, c, d, e]，而第二个列表是[d, e]，那么第一个列表是什么？', 'options': ['[a]', '[a, b]', '[a, b, c]', '[b, c]'], 'answer': 'D'}, {'question': '选择题 3：\n当第一个列表是[a, b, c]，新的列表是[a, b, c, d, e]，那么第二个列表是什么？', 'options': ['[d, e]', '[b, c, d]', '[c, d, e]', '[b, d, e]'], 'answer': 'C'}], 'id': 'round1_test_data_001'}, {'problem': '有一个列表，找出该列表的最后一个元素。\n\n下列选项中哪个是列表 `[a, b, c, d]` 

In [10]:
sorted_data

[{'problem': '有一群人和一些食物类型。下列是关于这些个体和食物的已知信息：\n\n1. 鸡肉是一种食物。\n2. 苹果是一种食物。\n3. 如果X吃了Y，且X活着，则Y是一种食物。\n4. Bill存活。\n5. Bill吃了花生。\n6. John吃所有食物。\n7. Sue吃所有Bill吃的食物。\n8. John喜欢所有食物。\n\n根据以上信息，回答以下选择题：',
  'questions': [{'question': '选择题 1：\n谁喜欢吃花生？',
    'options': ['Bill', 'Sue', 'John', 'None of the above'],
    'answer': '#'}],
  'id': 'round1_test_data_000'},
 {'problem': '设有两个列表操作，第一个列表中包含若干元素，并且可以将第二个列表的元素追加到第一个列表的末尾形成一个新的列表。根据这个操作，请回答以下问题：',
  'questions': [{'question': '选择题 1：\n当第一个列表是[a, b, c]，第二个列表是[d, e]时，新的列表是什么？',
    'options': ['[a, b, c, d]',
     '[a, d, e, b, c]',
     '[a, b, c, d, e]',
     '[d, e, a, b, c]'],
    'answer': 'A'},
   {'question': '选择题 2：\n如果新列表是[a, b, c, d, e]，而第二个列表是[d, e]，那么第一个列表是什么？',
    'options': ['[a]', '[a, b]', '[a, b, c]', '[b, c]'],
    'answer': 'D'},
   {'question': '选择题 3：\n当第一个列表是[a, b, c]，新的列表是[a, b, c, d, e]，那么第二个列表是什么？',
    'options': ['[d, e]', '[b, c, d]', '[c, d, e]', '[b, d, e]'],
    'answer': 'C'}],
  'id': 'round1_test_data_001'},


In [11]:
def find_missing_ids(dict_list):
    # 提取所有序号
    extracted_ids = {int(d['id'][-3:]) for d in dict_list}

    # 创建0-500的序号集合
    all_ids = set(range(500))

    # 找出缺失的序号
    missing_ids = all_ids - extracted_ids

    return sorted(missing_ids)


# 示例字典列表
dict_list = sorted_data

# 找出缺失的序号
missing_ids = find_missing_ids(dict_list)
print("缺失的序号:", missing_ids)

缺失的序号: [4, 12, 37, 70, 71, 95, 106, 135, 151, 220, 222, 233, 235, 237, 264, 267, 284, 292, 375, 394, 403, 410, 417, 422, 481]


In [12]:
len(missing_ids)

25

In [13]:
data = []
with open('round1_test_data.jsonl') as reader:
    for id, line in enumerate(reader):
        if (id in missing_ids):
            sample = json.loads(line)
            for question in sample['questions']:
                question['answer'] = 'A'
            sorted_data.append(sample)
sorted_data = sorted(sorted_data, key=lambda x: int(str(x['id'])[-3:]))


In [14]:
with open('upload-pipeline.jsonl', 'w') as writer:
    for sample in sorted_data:
        writer.write(json.dumps(sample, ensure_ascii=False))
        writer.write('\n')